In [ ]:
#tenant_folios

import pandas as pd
from datetime import datetime


contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\final_combined_contacts_data.xlsx')
tenant_contacts_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\tenant_contacts.xlsx')
properties_df = pd.read_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\properties.xlsx')


tenant_df = contacts_df[
    (contacts_df['Roles'] == 'Tenant') | 
    (contacts_df['Roles'] == 'Supplier, Tenant') | 
    (contacts_df['Roles'] == 'Owner, Supplier, Tenant') | 
    (contacts_df['Roles'] == 'Owner, Tenant')
].copy()

#
def remove_dollar_sign(value):
    if pd.isna(value):
        return None
    return float(str(value).replace('$', '').replace(',', '').strip())


def extract_bond_required(value):
    if pd.notna(value) and 'required,' in value:
        
        required_part = value.split('required,')[0].strip()
        return remove_dollar_sign(required_part)
    return None


tenant_folios_data = []

# Set initial ID value
id_counter = 1001


for index, row in tenant_df.iterrows():
    
    tenant_contact_row = tenant_contacts_df[tenant_contacts_df['reference'] == row['Reference']]
    tenant_contact_id = tenant_contact_row['id'].values[0] if not tenant_contact_row.empty else None

    
    property_row = properties_df[properties_df['reference'] == row['Inner_Property']]
    property_id = property_row['id'].values[0] if not property_row.empty else None

    
    bond_required = extract_bond_required(row['Inner_Bond'])

    
    agreement_split = row['Inner_Agreement'].split(' to ') if pd.notna(row['Inner_Agreement']) else [None, None]
    agreement_start = agreement_split[0].replace('from ', '') if agreement_split[0] else None
    agreement_end = agreement_split[1] if len(agreement_split) > 1 else None

    
    agreement_start = pd.to_datetime(agreement_start).strftime('%Y-%m-%d') if agreement_start else None
    agreement_end = pd.to_datetime(agreement_end).strftime('%Y-%m-%d') if agreement_end else None

    
    rent = remove_dollar_sign(row['Inner_Rent'])
    bond_held = remove_dollar_sign(row['Folio_Bond Held'])
    part_paid = remove_dollar_sign(row['Folio_Part Paid'])
    deposit = remove_dollar_sign(row['Folio_Deposits'])
    uncleared = remove_dollar_sign(row['Folio_Uncleared'])

    
    move_in = pd.to_datetime(row['Inner_Moved In']).strftime('%Y-%m-%d') if pd.notna(row['Inner_Moved In']) else None
    move_out = pd.to_datetime(row['Inner_Moved Out']).strftime('%Y-%m-%d') if pd.notna(row['Inner_Moved Out']) else None
    paid_to = pd.to_datetime(row['Inner_Paid To']).strftime('%Y-%m-%d') if pd.notna(row['Inner_Paid To']) else None

    
    folio_code = row['Inner_Tenant Folio Code'].split('(')[1].split(')')[0] if pd.notna(row['Inner_Tenant Folio Code']) else None


    periodic_tenancy = 1 if row['Inner_Periodic Tenancy'] == 'Yes' else 0

    
    tenant_folios_data.append({
        'id': id_counter,
        'tenant_contact_id': tenant_contact_id,
        'property_id': property_id,
        'rent': rent,
        'rent_type': row['Inner_Period'],
        'rent_includes_tax': 0,
        'bond_required': bond_required,
        'bond_held': bond_held,
        'move_in': move_in,
        'move_out': move_out,
        'agreement_start': agreement_start,
        'agreement_end': agreement_end,
        'periodic_tenancy': periodic_tenancy,
        'paid_to': paid_to,
        'part_paid': part_paid,
        'invoice_days_in_advance': 'NULL',
        'rent_review_frequency': 12,
        'next_rent_review': 'NULL',
        'exclude_form_arrears': 'NULL',
        'bank_reterence': row['Inner_Bank Reference'],
        'receipt_warning': row['Inner_Receipt Warning'],
        'tenant_access': 'NULL',
        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'updated_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'pro_rata_to': 'NULL',
        'rent_invoice': 'NULL',
        'bond_already_paid': 'NULL',
        'bond_receipted': 'NULL',
        'bond_arreas': 'NULL',
        'bond_reference': 'NULL',
        'break_lease': 'NULL',
        'termination': 'NULL',
        'notes': 'NULL',
        'status': 'true',
        'due': 'NULL',
        'total_payout': 'NULL',
        'advance_pay': 'NULL',
        'deposit': deposit,
        'money_in': 0,
        'money_out': 0,
        'folio_code': folio_code,
        'company_id': 1,
        'opening_balance': 'NULL',
        'bond_part_paid_description': 'NULL',
        'bond_due_date': 'NULL',
        'bond_cleared_date': 'NULL',
        'uncleared': uncleared,
        'previous': 0,
        'part_paid_description': 'NULL'
    })

    
    id_counter += 1


tenant_folios_df = pd.DataFrame(tenant_folios_data)


tenant_folios_df.to_excel(r'C:\Users\MSI\Documents\Migration Tool New\DB Tables\Contacts DB\tenant_folios.xlsx', index=False)

print("tenant_folios.xlsx created successfully!")
